In [22]:


import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

In [23]:
from datasets import load_dataset

dataset_pop = load_dataset(
    "csv",
    data_files="hf://datasets/pitehu/SimBench/SimBenchPop.csv"
)

dataset_grouped = load_dataset(
    "csv",
    data_files="hf://datasets/pitehu/SimBench/SimBenchGrouped.csv"
)

print(f"Righe in Pop: {len(dataset_pop['train'])}")
print(f"Righe in Grouped: {len(dataset_grouped['train'])}")

# --- CORREZIONE QUI SOTTO ---
# Accediamo a ['train'] e usiamo il metodo nativo .to_pandas()
df_pop = dataset_pop["train"].to_pandas()
df_grouped = dataset_grouped["train"].to_pandas()

print("\nStruttura SimBenchPop (prime 5 colonne):")
print(df_pop.columns.tolist()[:5])

print("\nStruttura SimBenchGrouped (prime 5 colonne):")
print(df_grouped.columns.tolist()[:5])

# Ora puoi verificare che la colonna esista davvero
if 'group_prompt_variable_map' in df_grouped.columns:
    print("\n✅ Colonna 'group_prompt_variable_map' trovata correttamente!")
else:
    print("\n❌ Colonna NON trovata.")

Righe in Pop: 7167
Righe in Grouped: 6343

Struttura SimBenchPop (prime 5 colonne):
['Unnamed: 0', 'group_prompt_template', 'group_prompt_variable_map', 'input_template', 'auxiliary']

Struttura SimBenchGrouped (prime 5 colonne):
['group_prompt_template', 'group_prompt_variable_map', 'input_template', 'input_variable_map', 'output_format']

✅ Colonna 'group_prompt_variable_map' trovata correttamente!


In [24]:
import pandas as pd

# 1. Definiamo la lista dei paesi europei
europe_countries = [
    'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
    'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia',
    'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
    'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg',
    'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia',
    'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia',
    'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine',
    'United Kingdom', 'Vatican City'
]


# Funzione per classificare
def get_generation_europe(map_str):
    try:
        data_dict = ast.literal_eval(map_str)

        # A. Filtro Paese
        country = data_dict.get('cntry')
        if country not in europe_countries:
            return None

        # B. Filtro Età
        age = data_dict.get('age_group') or data_dict.get('AGE')

        if age == '18-29':
            return 'Gen Z (18-29)'
        elif age == '30-49':
            return 'Millennials (30-49)'
        else:
            return None

    except (ValueError, SyntaxError):
        return None


# Assicuriamoci che df_grouped sia caricato correttamente nel tuo ambiente
# (Se non lo è, ricaricalo con pd.read_csv come fatto in precedenza)

# 2. Applichiamo la classificazione
df_grouped['European_Generation'] = df_grouped['group_prompt_variable_map'].apply(get_generation_europe)

# Filtriamo il dataset finale
df_final = df_grouped.dropna(subset=['European_Generation']).copy()

# --- 3. UNIONE COLONNE TEMPLATE ---
# Uniamo 'group_prompt_template' con 'input_template' per avere il contesto completo
# Aggiungiamo uno spazio o un separatore per leggibilità
if 'input_template' in df_final.columns:
    df_final['full_prompt_context'] = (
            df_final['group_prompt_template'].astype(str) +
            " | " +
            df_final['input_template'].astype(str)
    )
else:
    print("ATTENZIONE: Colonna 'input_template' non trovata. Uso solo 'group_prompt_template'.")
    df_final['full_prompt_context'] = df_final['group_prompt_template']

# --- 4. SELEZIONE E VISUALIZZAZIONE ---

# Cerchiamo la colonna delle opzioni di risposta (spesso 'target_text' o 'choices')
col_options = None
possibili_colonne_risposta = ['target_text', 'choices', 'options', 'answer_options']

for col in possibili_colonne_risposta:
    if col in df_final.columns:
        col_options = col
        break

# Creiamo la lista delle colonne da mostrare
cols_to_show = ['European_Generation', 'full_prompt_context']
if col_options:
    cols_to_show.append(col_options)

print(f"Totale righe trovate: {len(df_final)}")
print("\n--- ESEMPIO DEI DATI ESTRATTI (Prime 3 righe) ---")

# Impostiamo l'opzione per visualizzare tutto il testo senza tagliarlo
pd.set_option('display.max_colwidth', None)

for index, row in df_final[cols_to_show].head(3).iterrows():
    print(f"\n[GENERAZIONE]: {row['European_Generation']}")
    print(f"[CONTESTO COMPLETO]: {row['full_prompt_context']}")
    if col_options:
        print(f"[RISPOSTA/OPZIONI]: {row[col_options]}")
    print("-" * 50)

Totale righe trovate: 104

--- ESEMPIO DEI DATI ESTRATTI (Prime 3 righe) ---

[GENERAZIONE]: Millennials (30-49)
[CONTESTO COMPLETO]: The year is 2016. You are from Finland. You are in the 30-49 age group. | And can I just check have you ever lived with a partner, without being married to them (or in a civil union)?

Options:
(A): Yes
(B): No
[RISPOSTA/OPZIONI]: {'A': 'Yes', 'B': 'No'}
--------------------------------------------------

[GENERAZIONE]: Millennials (30-49)
[CONTESTO COMPLETO]: The year is 2016. You are from France. You are in the 30-49 age group. | And can I just check have you ever lived with a partner, without being married to them (or in a civil union)?

Options:
(A): Yes
(B): No
[RISPOSTA/OPZIONI]: {'A': 'Yes', 'B': 'No'}
--------------------------------------------------

[GENERAZIONE]: Gen Z (18-29)
[CONTESTO COMPLETO]: The year is 2016. You are from Austria. You are in the 18-29 age group. | Are you or have you ever been a member of a trade union or similar organi

In [25]:
import pandas as pd

# Assumiamo che 'df_final' sia già stato creato con lo script precedente.
# Se non lo hai in memoria, esegui prima il codice del passaggio precedente.

# 1. Creazione del DataFrame per la GEN Z (18-29)
df_gen_z = df_final[df_final['European_Generation'] == 'Gen Z (18-29)'].copy()

# 2. Creazione del DataFrame per i MILLENNIALS (30-49)
df_millennials = df_final[df_final['European_Generation'] == 'Millennials (30-49)'].copy()

# --- VERIFICA E STATISTICHE ---

print(f"✅ DataFrame Gen Z creato: {len(df_gen_z)} righe")
print(f"✅ DataFrame Millennials creato: {len(df_millennials)} righe")

# 3. Pulizia opzionale: teniamo solo le colonne utili per l'analisi
colonne_utili = ['European_Generation', 'full_prompt_context']

# Aggiungiamo la colonna risposte se esiste (es. 'choices' o 'target_text')
col_risposte = None
for col in ['choices', 'target_text', 'answer_options']:
    if col in df_final.columns:
        col_risposte = col
        colonne_utili.append(col)
        break

df_gen_z = df_gen_z[colonne_utili]
df_millennials = df_millennials[colonne_utili]

# --- VISUALIZZAZIONE DI UN CONFRONTO ---
print("\n--- ESEMPIO GEN Z ---")
pd.set_option('display.max_colwidth', 150)  # Limitiamo la larghezza per leggere meglio
print(df_gen_z.head(2).to_string(index=True))

print("\n--- ESEMPIO MILLENNIALS ---")
print(df_millennials.head(2).to_string(index=True))

✅ DataFrame Gen Z creato: 24 righe
✅ DataFrame Millennials creato: 80 righe

--- ESEMPIO GEN Z ---
     European_Generation                                                                                                                                                                                                         full_prompt_context                                              answer_options
1067       Gen Z (18-29)  The year is 2016. You are from Austria. You are in the 18-29 age group. | Are you or have you ever been a member of a trade union or similar organisation?\n\nOptions:\n(A): Yes, currently\n(B): Yes, previously\n(C): No  {'A': 'Yes, currently', 'B': 'Yes, previously', 'C': 'No'}
1071       Gen Z (18-29)  The year is 2016. You are from Belgium. You are in the 18-29 age group. | Are you or have you ever been a member of a trade union or similar organisation?\n\nOptions:\n(A): Yes, currently\n(B): Yes, previously\n(C): No  {'A': 'Yes, currently', 'B': 'Yes, previously

In [39]:
%load_ext autoreload
%autoreload 2
import importlib
import ast
from openai import OpenAI
import os
from genzelo.prompts import kb_prompt
import json

importlib.reload(kb_prompt)
from genzelo.prompts.kb_prompt import KB_PROMPT
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("MISTRALAI_API_KEY"),
    base_url="https://api.mistral.ai/v1/"
)

# Cerchiamo automaticamente il nome della colonna che contiene le opzioni
# (poiché nel passaggio precedente l'abbiamo salvata dinamicamente)
col_options = None
possibili_nomi = ['choices', 'target_text', 'answer_options']

# Controlliamo in uno dei due df (hanno le stesse colonne)
for col in possibili_nomi:
    if col in df_millennials.columns:
        col_options = col
        break

if col_options is None:
    print("⚠️ ERRORE: Non trovo la colonna delle opzioni. Controlla i nomi nel dataframe.")
else:
    print(f"✅ Colonna opzioni rilevata: '{col_options}'\n")


# --- FUNZIONE DI FORMATTAZIONE ---
def stampa_formato_llm(df, file_name):
    """
    Stampa le righe del dataframe nel formato specifico per il prompt engineering.
    """
    output = ""
    list_examples = []
    i = 1
    for index, row in df.iterrows():
        output += f"""{'-' * 50}start
[GENERAZIONE]: {row['European_Generation']}
[CONTESTO COMPLETO]: {row['full_prompt_context']}
[RISPOSTA/OPZIONI]: {row[col_options]}
{'-' * 50}finish
 """
        if i == 10:
            content = KB_PROMPT.format(documents=output, older_doc="[]")
            print("*" * 100)
            output = ""
            i = 1
            messages = [{"role": "user", "content": content}]
            completion = client.chat.completions.create(messages=messages, model="mistral-medium-latest")
            response = completion.choices[0].message.content
            response = response.replace("```json", "").replace("```", "")
            print(response)
            list_examples.extend(json.loads(response))
            print(f"size list_examples {len(list_examples)}")
        else:
            i = i + 1
    nome_file = f"{file_name}.json"

    with open(nome_file, "w", encoding="utf-8") as f:
        json.dump(list_examples, f, indent=4, ensure_ascii=False)

    print(f"\n🚀 Operazione completata! Salvati {len(list_examples)} esempi in '{nome_file}'.")


# --- ESECUZIONE ---

print("=== DATASET MILLENNIALS (Esempi) ===")
stampa_formato_llm(df_millennials, "millenials")

print("\n\n")  # Spazio tra i due blocchi

print("=== DATASET GEN Z (Esempi) ===")
stampa_formato_llm(df_gen_z, "genz")

[
    {
        "doc": "Gen Z in Italy (2016) demonstrates near-universal digital integration, with internet usage framed as a daily or near-daily expectation across devices (computers, tablets, smartphones). The normalization of 'every day' as a response option—without qualification—reflects a generation for whom constant connectivity is a baseline condition, not an exceptional behavior. This implies prioritization of digital accessibility in personal, educational, and professional spheres, with minimal distinction between device types or use cases.",
        "metadata": [
            "genz",
            "digital_behavior",
            "connectivity_norm",
            "Italy",
            "2016_context",
            "digital_native_era",
            "work_life_blurring"
        ]
    },

    {
        "doc": "Gen Z in Italy (2016) engages in consumer activism as a normalized form of civic participation, with boycotting products presented as a standard, measurable action for social or 